## AudioLIME Notebook

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")
warnings.filterwarnings("ignore", message="Trying to estimate tuning from empty frequency set.")


In [ ]:
import numpy as np
import librosa

def extract_full_features(y, sr):

    features = []

    # Chroma
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    features.extend([np.mean(chroma), np.var(chroma)])

    # RMS
    rms = librosa.feature.rms(y=y)[0]
    features.extend([np.mean(rms), np.var(rms)])

    # Spectral
    features.extend([
        np.var(librosa.feature.spectral_centroid(y=y, sr=sr)[0]),
        np.var(librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]),
        np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)[0]),
        np.var(librosa.feature.spectral_rolloff(y=y, sr=sr)[0]),
    ])

    # Zero-crossing
    zcr = librosa.feature.zero_crossing_rate(y)[0]
    features.extend([np.mean(zcr), np.var(zcr)])

    # Harmonic & Percussive
    harmonic, percussive = librosa.effects.hpss(y)
    features.extend([
        np.mean(harmonic), np.var(harmonic),
        np.mean(percussive), np.var(percussive)
    ])

    # Tempo
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    features.append(tempo)

    # MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    for coeff in mfcc:
        features.extend([np.mean(coeff), np.var(coeff)])

    return features



In [ ]:
import pickle
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from audioLIME.audioLIME import lime_audio, factorization_spleeter
from IPython.display import Audio, display
import librosa
import numpy as np

In [ ]:
# Load audio
file_path = "../data/raw/blues.00000.wav"
y, sr = librosa.load(file_path, sr=22050)

print("Audio:", y.shape, y.dtype)

In [ ]:
# Load model
with open("../models/svm_genre_classifier.pkl", "rb") as f:
    model_bundle = pickle.load(f)

scaler = model_bundle["scaler"]
svm = model_bundle["model"]

print("Scaler expects:", scaler.n_features_in_)
print("Single audio features shape:",len(extract_full_features(y,sr)))

In [ ]:
from scipy.special import softmax
import numpy as np

def svm_predict(audio_input):
    if isinstance(audio_input, np.ndarray):
        if audio_input.ndim == 1:
            audio_input = [audio_input]

    features = []

    for audio in audio_input:
        if len(audio) < 2048:
            audio = np.pad(audio, (0, 2048 - len(audio)))

        f = extract_full_features(audio, sr)
        features.append(f)

    features = np.array(features)
    scaled = scaler.transform(features)

    if len(svm.classes_) == 2:
        decision = svm.decision_function(scaled)
        decision = np.vstack([-decision, decision]).T
    else:
        decision = svm.decision_function(scaled)

    probs = softmax(decision, axis=1)
    print(f"[DEBUG] svm_predict returned shape: {probs.shape}")

    return probs

In [ ]:
total_duration = len(y) / sr
segment_length_seconds = 3.0

n_segments = max(1, int(total_duration / segment_length_seconds))

if y.ndim == 1:
    y = np.stack([y, y], axis=-1)  # 2D shape (samples, channels)

# Factorize
factorizer = factorization_spleeter.SpleeterFactorization(
    input=y,
    temporal_segmentation_params={
        "type": "fixed_length",
        "segment_length": segment_length_seconds,
        "n_temporal_segments": n_segments
    },
    composition_fn=np.sum,
    target_sr=sr
)

print(f"Segment length: {segment_length_seconds:.2f}s, Segments: {n_segments}")


In [ ]:
# Explain with audioLIME
explainer = lime_audio.LimeAudioExplainer(random_state=42)
explanation = explainer.explain_instance(
    factorization=factorizer,
    predict_fn=svm_predict,
    top_labels=1,
    num_features=10,
    num_samples=1000
)


In [ ]:
# Visualize
components, indices = explanation.get_sorted_components(
    label=explanation.top_labels[0],
    num_components=5,
    return_indeces=True
)

for i, comp in enumerate(components):
    print(f"Component {indices[i]}")
    display(Audio(comp, rate=sr))